In [4]:
import os 
slack_token = os.environ.get('SLACK_TOKEN')

In [5]:
from llama_index import GPTListIndex, SlackReader
slack_token = os.getenv("SLACK_TOKEN")
channel_ids = ["hpc"]
documents = SlackReader(slack_token=slack_token).load_data(channel_ids=channel_ids)
index = GPTListIndex.from_documents(documents)

KeyboardInterrupt: 

In [ ]:
from slack_sdk import WebClient
client = WebClient(token=slack_token)
res = client.api_test()
if not res["ok"]:
    raise ValueError(f"Error initializing Slack API: {res['error']}")

In [ ]:
request = client.api_call("users.list")
request['members'][3]

{'id': 'W8FRV3WSD',
 'team_id': 'T011UK0B5QD',
 'name': 'parekhr',
 'deleted': False,
 'color': 'aba727',
 'real_name': 'Ruchi Parekh',
 'tz': 'America/New_York',
 'tz_label': 'Eastern Daylight Time',
 'tz_offset': -14400,
 'profile': {'title': 'Director, Annotation & Analytics',
  'phone': '+1 (571) 209-4000 x1444',
  'skype': '',
  'real_name': 'Ruchi Parekh',
  'real_name_normalized': 'Ruchi Parekh',
  'display_name': 'parekhr',
  'display_name_normalized': 'parekhr',
  'fields': None,
  'status_text': '',
  'status_emoji': '',
  'status_emoji_display_info': [],
  'status_expiration': 1604379599,
  'avatar_hash': '4055dd671afd',
  'image_original': 'https://avatars.slack-edge.com/2019-01-18/527300937698_4055dd671afd83c47254_original.jpg',
  'is_custom_image': True,
  'first_name': 'Ruchi',
  'last_name': 'Parekh',
  'image_24': 'https://avatars.slack-edge.com/2019-01-18/527300937698_4055dd671afd83c47254_24.jpg',
  'image_32': 'https://avatars.slack-edge.com/2019-01-18/527300937698_4

In [ ]:
user_map = {}
request = client.api_call("users.list")
if request['ok']:
    for item in request['members']:
        id = item['id']
        username = item['name']
        name = item['profile']['real_name']
        user_map[id] = name

print(f"{len(user_map)} users")

171 users


In [4]:

from typing import List, Optional
import time
import logging
logger = logging.getLogger()

def _read_message(channel_id: str, message_ts: str) -> str:
    from slack_sdk.errors import SlackApiError

    """Read a message."""

    messages_text: List[str] = []
    next_cursor = None
    while True:
        try:
            # https://slack.com/api/conversations.replies
            # List all replies to a message, including the message itself.
            result = client.conversations_replies(
                channel=channel_id, ts=message_ts, cursor=next_cursor
            )
            messages = result["messages"]
            messages_text.extend(message["text"] for message in messages)
            if not result["has_more"]:
                break

            next_cursor = result["response_metadata"]["next_cursor"]
        except SlackApiError as e:
            if e.response["error"] == "ratelimited":
                logger.error(
                    "Rate limit error reached, sleeping for: {} seconds".format(
                        e.response.headers["retry-after"]
                    )
                )
                time.sleep(int(e.response.headers["retry-after"]))
            else:
                logger.error("Error parsing conversation replies: {}".format(e))

    return "\n\n".join(messages_text)

def _read_channel(channel_id: str, reverse_chronological: bool) -> str:
    from slack_sdk.errors import SlackApiError

    """Read a channel."""

    result_messages: List[str] = []
    next_cursor = None
    while True:
        try:
            # Call the conversations.history method using the WebClient
            # conversations.history returns the first 100 messages by default
            # These results are paginated,
            # see: https://api.slack.com/methods/conversations.history$pagination
            conversations_history_kwargs = {
                "channel": channel_id,
                "cursor": next_cursor,
            }
            result = client.conversations_history(
                **conversations_history_kwargs  # type: ignore
            )
            conversation_history = result["messages"]
            # Print results
            logger.info(
                "{} messages found in {}".format(
                    len(conversation_history), channel_id
                )
            )
            result_messages.extend(
                _read_message(channel_id, message["ts"])
                for message in conversation_history
            )
            if not result["has_more"]:
                break
            next_cursor = result["response_metadata"]["next_cursor"]

        except SlackApiError as e:
            if e.response["error"] == "ratelimited":
                logger.error(
                    "Rate limit error reached, sleeping for: {} seconds".format(
                        e.response.headers["retry-after"]
                    )
                )
                time.sleep(int(e.response.headers["retry-after"]))
            else:
                logger.error("Error parsing conversation replies: {}".format(e))

    return (
        "\n\n".join(result_messages)
        if reverse_chronological
        else "\n\n".join(result_messages[::-1])
    )




In [7]:
channel_id = "C019ZPNLLA0" # workflows
channel_content = _read_channel(
    channel_id, reverse_chronological=True
)


'Noticed by <@W94TA6021> and posted on <#C03CZHVDQF4|go> channel but probably even more pertinent to this channel since it hasn\'t been mentioned AFAIK but is used by a number of companies for ML and data processing: <https://docs.flyte.org/en/latest/>\n<https://github.com/flyteorg/flyte>\n\nDuring a volume EM data workgroup meeting (most participants based in Europe), a researcher working on a VDI solution for bioimage analysis talked about how the more "computationally experienced" people were using NextFlow for their workflows.   This BAND system is an experiment to test general computational access with virtual workstation display via VNC.\n\nI mention this because there was a question on how much NextFlow is being adopted for bioimaging in addition to its current bioinformatics concentration.\n\nWe\'ve started discussing with EMBL and others on using Nextflow for bioimaging pipelines. There\'s an ongoing chat here: <https://imagesc.zulipchat.com/#narrow/stream/367292-Nextflow>\n\n

In [8]:
print(channel_content)

Noticed by <@W94TA6021> and posted on <#C03CZHVDQF4|go> channel but probably even more pertinent to this channel since it hasn't been mentioned AFAIK but is used by a number of companies for ML and data processing: <https://docs.flyte.org/en/latest/>
<https://github.com/flyteorg/flyte>

During a volume EM data workgroup meeting (most participants based in Europe), a researcher working on a VDI solution for bioimage analysis talked about how the more "computationally experienced" people were using NextFlow for their workflows.   This BAND system is an experiment to test general computational access with virtual workstation display via VNC.

I mention this because there was a question on how much NextFlow is being adopted for bioimaging in addition to its current bioinformatics concentration.

We've started discussing with EMBL and others on using Nextflow for bioimaging pipelines. There's an ongoing chat here: <https://imagesc.zulipchat.com/#narrow/stream/367292-Nextflow>

We also have 

In [61]:

channel_id = "C057Z7J7F29"
message_ts = "1684259866.667699"

res = client.chat_getPermalink(channel=channel_id, message_ts=message_ts)
if res['ok']:
    print(res['permalink'])


https://janelia-dev.slack.com/archives/C057Z7J7F29/p1684259866667699
